<a href="https://colab.research.google.com/github/brokoli777/Emotion-Recogntion/blob/main/Retry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import kagglehub

# path = kagglehub.dataset_download("msambare/fer2013")

# print("Path to dataset files:", path)

100%|██████████| 60.3M/60.3M [00:01<00:00, 49.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/msambare/fer2013/versions/1


In [3]:
# import shutil

# # Define source and destination paths
# source = "/root/.cache/kagglehub/datasets/msambare/fer2013/versions/1"
# destination = "/content/dataset"

# # Move files to destination folder
# shutil.move(source, destination)

# print(f"Files moved to {destination}")

Files moved to /content/dataset


In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Input, Dense, Conv2D, Dropout, Flatten, MaxPooling2D

In [3]:
TRAIN_DIR = './dataset/test'
TEST_DIR = './dataset/train'

In [4]:
print(os.listdir(TRAIN_DIR))
print(os.listdir(TEST_DIR))

['surprise', 'angry', 'disgust', 'fear', 'happy', 'neutral', 'sad']
['surprise', 'angry', 'disgust', 'fear', 'happy', 'neutral', 'sad']


In [5]:
print(len(os.listdir(TRAIN_DIR+'/angry')))
print(len(os.listdir(TRAIN_DIR+'/disgust')))

958
111


In [6]:
def load_dataset(directory):
    image_paths = []
    labels = []

    for label in os.listdir(directory):
        label_path = os.path.join(directory, label)  # Use os.path.join
        if not os.path.isdir(label_path):  # Skip if it's not a directory
            continue

        for filename in os.listdir(label_path):  # Correctly use label_path
            image_path = os.path.join(label_path, filename)
            image_paths.append(image_path)
            labels.append(label)

        print(label, "Completed")

    return image_paths, labels

In [7]:
## convert into dataframe
train = pd.DataFrame()
train['image'], train['label'] = load_dataset(TRAIN_DIR)
# shuffle the dataset
train = train.sample(frac=1).reset_index(drop=True)
train.head()

surprise Completed
angry Completed
disgust Completed
fear Completed
happy Completed
neutral Completed
sad Completed


,image,label
0,./dataset/test/surprise/PrivateTest_58522921.jpg,surprise
1,./dataset/test/neutral/PublicTest_14790723.jpg,neutral
2,./dataset/test/happy/PrivateTest_2626531.jpg,happy
3,./dataset/test/sad/PrivateTest_9859012.jpg,sad
4,./dataset/test/sad/PrivateTest_44238830.jpg,sad


In [8]:
test = pd.DataFrame()
test['image'], test['label'] = load_dataset(TEST_DIR)
test.head()

surprise Completed
angry Completed
disgust Completed
fear Completed
happy Completed
neutral Completed
sad Completed


,image,label
0,./dataset/train/surprise/Training_49926859.jpg,surprise
1,./dataset/train/surprise/Training_62427852.jpg,surprise
2,./dataset/train/surprise/Training_34478752.jpg,surprise
3,./dataset/train/surprise/Training_61865521.jpg,surprise
4,./dataset/train/surprise/Training_71645335.jpg,surprise


In [9]:
def get_features(images):
    features = []
    for image in images:
        img = load_img(image, color_mode='grayscale')
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

In [10]:
train_features = get_features(train['image'])
test_features = get_features(test['image'])

In [11]:
## normalize the image
x_train = train_features/255.0
x_test = test_features/255.0

In [12]:
## convert label to integer
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()
encode.fit(train['label'])
y_train = encode.transform(train['label'])
y_test = encode.transform(test['label'])

In [13]:
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [14]:
input_shape = (48, 48, 1)
output_class = 7

In [15]:
model = Sequential()

model.add(Input(input_shape))

model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())


model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(output_class, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
emotion_model_info = model.fit(
        x=x_train,
        y=y_train,
        batch_size=128,
        epochs=100,
        verbose=1,
        validation_data=(x_test, y_test))

Epoch 1/100
 8/57 ━━━━━━━━━━━━━━━━━━━━ 4:33 6s/step - accuracy: 0.1394 - loss: 1.9463

KeyboardInterrupt: 

In [ ]:
model.evaluate(x_test, y_test, verbose=1)